In [11]:
import pandas as pd
import os

In [12]:
columns_to_keep=['tpep_pickup_datetime','PULocationID','DOLocationID','passenger_count','trip_distance','total_amount','tip_amount','congestion_surcharge']

taxi_total=pd.read_csv('2021_Yellow_Taxi_Trip_Data_20240207.csv',usecols=columns_to_keep)

In [13]:
taxi_total.head(10)

,tpep_pickup_datetime,passenger_count,trip_distance,PULocationID,DOLocationID,tip_amount,total_amount,congestion_surcharge
0,01/01/2021 12:30:10 AM,1.0,2.10,142,43,0.00,11.80,2.5
1,01/01/2021 12:51:20 AM,1.0,0.20,238,151,0.00,4.30,0.0
2,01/01/2021 12:43:30 AM,1.0,14.70,132,165,8.65,51.95,0.0
3,01/01/2021 12:15:48 AM,0.0,10.60,138,132,6.05,36.35,0.0
4,01/01/2021 12:31:49 AM,1.0,4.94,68,33,4.06,24.36,2.5
5,01/01/2021 12:16:29 AM,1.0,1.60,224,68,2.35,14.15,2.5
6,01/01/2021 12:00:28 AM,1.0,4.10,95,157,0.00,17.30,0.0
7,01/01/2021 12:12:29 AM,1.0,5.70,90,40,0.00,21.80,2.5
8,01/01/2021 12:39:16 AM,1.0,9.10,97,129,0.00,28.80,0.0
9,01/01/2021 12:26:12 AM,2.0,2.70,263,142,3.15,18.95,2.5


In [14]:
# 记录原始行数
original_row_count = len(taxi_total)

# 排除PULocationID, DOLocationID, trip_distance, total_amount中含有NaN的行
taxi_total = taxi_total.dropna(subset=['PULocationID', 'DOLocationID', 'trip_distance', 'total_amount'])

# 然后，排除PULocationID和DOLocationID不在1到263范围内的行
taxi_total = taxi_total[(taxi_total['PULocationID'] >= 1) & (taxi_total['PULocationID'] <= 263)]
taxi_total = taxi_total[(taxi_total['DOLocationID'] >= 1) & (taxi_total['DOLocationID'] <= 263)]

# 最后，排除trip_distance值为0的行
taxi_total = taxi_total[taxi_total['trip_distance'] > 0]

# 将passenger_count为NaN或者0的行改为1
taxi_total['passenger_count'] = taxi_total['passenger_count'].fillna(0).apply(lambda x: 1 if x == 0 else x)

# 计算删除的行数
deleted_row_count = original_row_count - len(taxi_total)

# 输出删除了多少行
print(f"Deleted {deleted_row_count} rows.")

Deleted 885755 rows.


In [15]:
# 转换 tpep_pickup_datetime 列为 datetime 类型
taxi_total['tpep_pickup_datetime'] = pd.to_datetime(taxi_total['tpep_pickup_datetime'], format='%m/%d/%Y %I:%M:%S %p')
taxi_total['tpep_pickup_datetime']

0          2021-01-01 00:30:10
1          2021-01-01 00:51:20
2          2021-01-01 00:43:30
3          2021-01-01 00:15:48
4          2021-01-01 00:31:49
                   ...        
30904067   2021-12-31 23:18:02
30904068   2021-12-31 23:46:00
30904069   2021-12-31 23:46:00
30904070   2021-12-31 23:53:00
30904071   2021-12-31 23:19:28
Name: tpep_pickup_datetime, Length: 30018317, dtype: datetime64[ns]

In [16]:
taxi_total = taxi_total[taxi_total['tpep_pickup_datetime'].dt.year == 2021]
taxi_total

,tpep_pickup_datetime,passenger_count,trip_distance,PULocationID,DOLocationID,tip_amount,total_amount,congestion_surcharge
0,2021-01-01 00:30:10,1.0,2.10,142,43,0.00,11.80,2.5
1,2021-01-01 00:51:20,1.0,0.20,238,151,0.00,4.30,0.0
2,2021-01-01 00:43:30,1.0,14.70,132,165,8.65,51.95,0.0
3,2021-01-01 00:15:48,1.0,10.60,138,132,6.05,36.35,0.0
4,2021-01-01 00:31:49,1.0,4.94,68,33,4.06,24.36,2.5
...,...,...,...,...,...,...,...,...
30904067,2021-12-31 23:18:02,1.0,3.99,145,107,5.80,26.52,NaN
30904068,2021-12-31 23:46:00,1.0,5.96,236,158,2.85,29.20,NaN
30904069,2021-12-31 23:46:00,1.0,4.99,48,13,5.23,29.50,NaN
30904070,2021-12-31 23:53:00,1.0,1.36,75,75,2.00,16.00,NaN


In [8]:
def month_to_season(month):
    if month in [3, 4, 5]:
        return 'spring'
    elif month in [6, 7, 8]:
        return 'summer'
    elif month in [9, 10, 11]:
        return 'autumn'
    else:
        return 'winter'

In [10]:
# 创建输出目录
output_dir = 'split_data_by_seasons'
os.makedirs(output_dir, exist_ok=True)

# 拆分数据并保存
for (month, is_weekend), group in taxi_total.groupby([
    taxi_total['tpep_pickup_datetime'].dt.month, 
    taxi_total['tpep_pickup_datetime'].dt.weekday >= 5
]):
    season = month_to_season(month)  # 使用月份获取季节
    day_type = 'weekends' if is_weekend else 'weekdays'
    for hour, hour_group in group.groupby(group['tpep_pickup_datetime'].dt.hour):
        # 文件名现在包含季节而不是月份
        filename = f'taxi_{season}_{day_type}_{hour:02d}.csv'
        filepath = os.path.join(output_dir, filename)
        
        # 检查文件是否存在，以确定是否需要写入头部
        if not os.path.isfile(filepath):
            hour_group.to_csv(filepath, mode='w', header=True, index=False)
        else:
            hour_group.to_csv(filepath, mode='a', header=False, index=False)

